In [1]:
import pandas as pd

#stampare i dati
pd.read_excel("file_di_dati.xls", sheet_name="porzioni di viaggio") 
pd.read_excel("file_di_dati.xls", sheet_name="tempi di percorrenza a vuoto") 

#salvare i dati dentro una variabile
xlsx1 = pd.read_excel("file_di_dati.xls", sheet_name="porzioni di viaggio") 
xlsx2 = pd.read_excel("file_di_dati.xls", sheet_name="tempi di percorrenza a vuoto") 

#filtrare e salvare solo alcune colonne dentro una variabile
#in questo caso verranno selezionate la colonna A e le colonne dalla C alla E
SP_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='C')          #colonna stazione partenza
SP_i=SP_i.values.tolist();
SP_i.pop(0);
SP_i.pop();

SA_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='D')          #colonna stazione arrivo
SA_i=SA_i.values.tolist();
SA_i.pop(0);
SA_i.pop();

S_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='E')          #colonna tempi inizio
S_i=S_i.values.tolist();
S_i.pop(0);
S_i.pop();

A_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='F')           #colonna tempi fine
A_i=A_i.values.tolist();
A_i.pop(0);
A_i.pop();

P_extra = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='B')        #colonna passeggeri extra
P_extra = P_extra.values.tolist();
P_extra.pop(0);
P_extra.pop();

#P_extra
#xlsx1 #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#xlsx2 #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#S_i #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#A_i #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#SP_i
#SA_i


In [2]:
import time
stazioni=(xlsx2.values.tolist())[0]
stazioni.pop(0);
for i in stazioni:
    i.rstrip();

DizionarioStazioni = {stazioni[i]:i for i in range(16)}

p=len(SP_i);

for i in range(p):
    stringa=SP_i[i][0];
    stringa.rstrip(); 
    SP_i[i][0]=DizionarioStazioni[stringa]
    stringa=SA_i[i][0];
    stringa.rstrip();
    SA_i[i][0]=DizionarioStazioni[stringa]
    
#riempire matrice tratte
tempi_vuoto = xlsx2.values.tolist(); #trasformo tabella in matrice

tempi_vuoto.pop(0); #elimino prima riga matrice (nomi fermate)
for i in range(16):
    tempi_vuoto[i].pop(0); #elimino prima colonna di ogni riga della matrice (nomi fermate)
for i in range(16):
    tempi_vuoto[i][i]=0;
for i in range(16):
    for j in range(16):
        tempi_vuoto[i][j]=tempi_vuoto[j][i];

In [ ]:
import mip
import datetime
import networkx as nx

m = mip.Model()

# >>> FORMULAZIONE >>>

k=4 #numero bus sostitutivi

P=[]
for i in range(p):
    P.append(P_extra[i][0])
P.insert(0,0)
P.append(0)

#Nodi
Di = 0
Df = p+2

V=[Di]
for x in range(p):
    V.append(x+1)
V.append(Df)



#archi
A=[]
for i in range(p):
    for j in range(p):
        if i != j and A_i[i][0].timestamp() + datetime.timedelta(minutes = tempi_vuoto[SA_i[i][0]][SP_i[j][0]]).seconds <= S_i[j][0].timestamp():
            A.append((i+1,j+1))

for i in range(p+1):
    A.append((0, i+1))
for i in range(p+1):    
    A.append((i+1, p+2))
A.append((0,p+2))


#Variabili (percorro arco / non percorro arco)
X = [dict() for b in range(k)]
for b in range(k):
    X[b]={(i,j): m.add_var(var_type = mip.BINARY) for (i,j) in A}

    
# VINCOLI

#vincolo: max 1 bus per tratta

for (i,j) in A:
    m.add_constr(mip.xsum(X[b][(i,j)] for b in range(k))<= 1); 
    
#vincolo: flusso entrante = flusso uscente
for b in range(k):
    for i in V:
        if i==0:
            m.add_constr(mip.xsum(X[b][i,j] for j in V if (i,j) in A) -   # Outgoing flow from i
                         mip.xsum(X[b][j,i] for j in V if (j,i) in A) ==  # Incoming flow into i
                         1) #1= num bus
        if i==p+2:
            m.add_constr(mip.xsum(X[b][i,j] for j in V if (i,j) in A) -   # Outgoing flow from i
                         mip.xsum(X[b][j,i] for j in V if (j,i) in A) ==  # Incoming flow into i
                         -1)
        if i!=0 and i!=p+2:
            m.add_constr(mip.xsum(X[b][i,j]  for j in V if (i,j) in A) -   # Outgoing flow from i
                         mip.xsum(X[b][j,i]  for j in V if (j,i) in A) ==  # Incoming flow into i
                         0)

#Funzione obiettivo

m.objective = mip.maximize(mip.xsum(P[i] * X[b][i,j] for (i,j) in A for b in range(k)))

#soluzione
status = m.optimize()
print("TOTALE PASSEGGERI RACCOLTI: ",m.objective_value)
for b in range(k):
    for (i,j) in A:
        if(X[b][i,j].x):
            print("prendo l'arco",(i,j))
            if(i!=0 and j!=p+2):
                print("Copro la tratta",i-1, 
                      "che parte dalla stazione\"", stazioni[SP_i[i-1][0]],
                      "\"e arriva alla stazione\"", stazioni[SA_i[i-1][0]],"\"")          
            else:
                if(i==0):
                    print("Copro la tratta deposito - ", stazioni[SP_i[j-1][0] if j!= p+2 else "deposito"])
                if(j==p+2):
                    print("Copro la tratta ", stazioni[SP_i[i-1][0]] if i!= 0 else "deposito","- deposito")
        




In [ ]:
m.write('model_graph.lp')
S_1_prova=[]
S_i_prova=[]
for i in S_i:
    S_i_prova.append(i[0].timestamp())
print(S_i_prova)
time_change = datetime.timedelta(minutes=10)
for i in S_i:
    S_1_prova.append(i[0].timestamp() + time_change.seconds)
print(S_1_prova)

In [59]:
print(A)

[(1, 3), (1, 4), (1, 5), (2, 5), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7)]
